### Selecting and preparing TIFF stacks for the website

In [ ]:
import os
import re
import sys
import glob
import json
import nrrd
import cv2
import vispy
import tifffile
import skimage
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors

from skimage import feature
from skimage import morphology

In [ ]:
# HACK: path to local image-utils package (which is currently only on box)
sys.path.append('/Users/keith.cheveralls/projects/opencell/')
from opencell.imaging import viz, utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# targets for demo
target_names = ['ATL2', 'ATL3', 'LAMP1', 'MTOR', 'POLR1C', 'RAB14', 'SEC13', 'TAF12', 'VAPA', 'CLTA']

### Aside: generating z-projections to quickly select 'good' FOVs for each target

In [ ]:
def process_directory(root_dirpath_in):
    '''
    Generate projections of all TIFFs in all subdirectories of the root directory
    
    *** This method assumes that the TIFF stack dimension order is (z, channel, x, y) ***
    '''
    
    # drop trailing slashes
    root_dirpath_in, _ = os.path.split(root_dirpath_in)    
    root_dirpath_out = '%s_PROC' % root_dirpath_in
    
    for dirpath_in, subdirs, filenames in os.walk(root_dirpath_in):
        dirpath_out = dirpath_in.replace(root_dirpath_in, root_dirpath_out)
    
        for filename in filenames:
            ext = '.%s' % filename.split('.')[-1]
            if ext!='.tif':
                continue
            
            # HACK: only process FOVs corresponding to the targets 
            # we plan to use for the open-cell demo
            of_interest = False
            for name in target_names:
                if name in filename:
                    of_interest = True
                    
            if not of_interest:
                continue
            
            print('Processing %s' % filename)
            os.makedirs(dirpath_out, exist_ok=True)
            stack = utils.load(os.path.join(dirpath_in, filename))
            
            # make DAPI grayscale projection
            filepath = os.path.join(dirpath_out, filename.replace(ext, '_DAPI.tif'))
            if not os.path.isfile(filepath):
                im_dapi = utils.autogain(stack[:, 0, :, :].max(axis=0))
                tifffile.imsave(filepath, im_dapi)
                
            # make an RGB projection (DAPI in gray, GFP in green)
            filepath = os.path.join(dirpath_out, filename.replace(ext, '_RGB.tif'))
            if not os.path.isfile(filepath):
                proj_rgb = viz.make_rgb(imb=stack[:, 0, :, :], im_bg=stack[:, 1, :, :])
                tifffile.imsave(filepath, proj_rgb)

In [ ]:
process_directory('/Volumes/keith-external/Plate_Microscopy/mNG96wp3_thawed/')

In [ ]:
process_directory('/Users/keith.cheveralls/image-data/pipeline/mNG96wp1_Thawed/')

### Writing stacks as NRRD files

In [ ]:
# manually selected FOVs for each target (by Keith in early Augst 2019)
# note that, for all FOVs, a 512x512 ROI was cropped from the top left to generate the NRRD files
p1 = '/Volumes/keith-external/Plate_Microscopy/mNG96wp1_Thawed/'
p3 = '/Volumes/keith-external/Plate_Microscopy/mNG96wp3_Thawed/'
p1_clones = '/Volumes/keith-external/Plate_Microscopy/mNG96wp1_Clones/'

kc_fovs_for_demo = {
    
    # plate1 clones
    'ATL2': os.path.join(p1_clones, 'ML0156_20190703_IJClean/A1-4_8_ATL2_PyProcessed_IJClean.tif'),
    'ATL3': os.path.join(p1_clones, 'ML0156_20190703_IJClean/A2-1_5_ATL3_PyProcessed_IJClean.tif'), 
    
    # plate1 thawed
    'MTOR': os.path.join(p1, 'ML0147_20190621_IJClean/D7_9_MTOR_PyProcessed_IJClean.tif'), 
    'VAPA': os.path.join(p1, 'ML0147_20190621_IJClean/H11_11_VAPA_PyProcessed_IJClean.tif'), 
    'RAB14': os.path.join(p1, 'ML0155_20190627_IJClean/E7_19_RAB14_PyProcessed_IJClean.tif'),
    'CLTA': os.path.join(p1, 'ML0143_20190612_IJClean/C12_6_CLTA_PyProcessed_IJClean.tif'),
    
    # plate3 thawed
    'POLR1C': os.path.join(p3, 'ML0158_20190709_IJClean/A3_1_POLR1C_PyProcessed_IJClean.tif'), 
    'SEC13': os.path.join(p3, 'ML0158_20190709_IJClean/F3_1_SEC13_PyProcessed_IJClean.tif'),
    'TAF12': os.path.join(p3, 'ML0164_20190718_IJClean/B1_17_TAF12_PyProcessed_IJClean.tif'), 
    'LAMP1': os.path.join(p3, 'ML0163_20190716_IJClean/E1_10_LAMP1_PyProcessed_IJClean.tif'),
    
}

In [ ]:
# manually selected FOVs by Manu (on 2019-08-16)
# order of parameters is (filepath, top-left-x, top-left-y, z-step)

ml_fovs_for_demo = {
    'ATL2': ('mNG96wp1_Clones/ML0156_20190703_IJClean/A1-1_7_ATL2_PyProcessed_IJClean.tif', 200, 199, 0.2),
    'ATL3': ('mNG96wp1_Clones/ML0156_20190703_IJClean/A2-2_7_ATL3_PyProcessed_IJClean.tif', 404, 405, 0.2),
    'RAB14': ('mNG96wp1//PublicationQuality/p1E7_1_RAB14_PyProcessed_IJClean.tif', 0, 133, 0.5),
    'CLTA': ('mNG96wp1//PublicationQuality/p1C12_5_CLTA_PyProcessed_IJClean.tif', 304, 202, 0.5),
    'LAMP1': ('mNG96wp3//PublicationQuality/p03E1_10_LAMP1_PyProcessed_IJClean.tif', 286, 167, 0.5),
    'MTOR': ('mNG96wp1//PublicationQuality/p1D7_6_MTOR_PyProcessed_IJClean.tif', 322, 180, 0.5),
    'POLR1C': ('mNG96wp3//PublicationQuality/p03A3_4_POLR1C_PyProcessed_IJClean.tif', 0, 0, 0.5),
    'SEC13': ('mNG96wp3//PublicationQuality/p03F4_4_PREB_PyProcessed_IJClean.tif', 153, 147, 0.5),
    'TAF12': ('mNG96wp3//PublicationQuality/p03B1_1_TAF12_PyProcessed_IJClean.tif', 162, 361, 0.5),
    'VAPA': ('mNG96wp1//PublicationQuality/p1H11_5_VAPA_PyProcessed_IJClean.tif', 152, 328, 0.5),
    'LMNA': ('mNG96wp3//PublicationQuality/p03E3_9_LAPTM4A_PyProcessed_IJClean.tif', 48, 303, 0.5),
    'LMNB1': ('mNG96wp3//PublicationQuality/p03E5_4_LMNB1_PyProcessed_IJClean.tif', 251, 330, 0.5),
    'POLR1A': ('mNG96wp3//PublicationQuality/p03A1_5_POLR1A_PyProcessed_IJClean.tif', 50, 61, 0.5),
    'POLR1C': ('mNG96wp3//PublicationQuality/p03A4_3_POLR1D_PyProcessed_IJClean.tif', 181, 61, 0.5),
    'MAP4': ('mNG96wp2//PublicationQuality/p02H1_10_MAP4_PyProcessed_IJClean.tif', 92, 276, 0.5),
    'CSNK2A1': ('mNG96wp3//PublicationQuality/p03B4_1_CSNK2A2_PyProcessed_IJClean.tif', 308, 86, 0.5),
    'CSNK2A2': ('mNG96wp3//PublicationQuality/p03B3_1_CSNK2A1_PyProcessed_IJClean.tif', 181, 49, 0.5),
    'TFRC': ('mNG96wp3//PublicationQuality/p03D7_6_TFRC_PyProcessed_IJClean.tif', 120, 63, 0.5),
    'SEC23A': ('mNG96wp3//PublicationQuality/p03E7_2_SEC23A_PyProcessed_IJClean.tif', 114, 419, 0.5),
    'POLR1D': ('mNG96wp3//PublicationQuality/p03A3_4_POLR1C_PyProcessed_IJClean.tif', 0, 0, 0.5),
    'ATP2B1': ('mNG96wp3//PublicationQuality/p03D1_10_ATP2B1_PyProcessed_IJClean.tif', 87, 93, 0.5),
    'TOP2A': ('mNG96wp3//PublicationQuality/p03B5_6_TOP2A_PyProcessed_IJClean.tif', 310, 17, 0.5),
}

In [ ]:
im = utils.load(fovs_for_demo['ATL2'])

In [ ]:
def to_nrrd(filepath_in, dirpath_out, roi_top_left, z_step_size, write_files=False):
    '''
    Save a pipeline plate microscopy TIFF stack as a set of NRRD files (one for each channel)
    *** assumes order of dimensions is (z, channel, x, y)
    
    roi_top_left: tuple of (row_num, col_num) identifying the top left corner of the ROI to crop
    '''
    
    # hard-coded xy pixel size in microns
    pixel_size = 0.2
    
    # hard-coded ROI size
    roi_shape = (600, 600)
    
    # extract fileroot - assumes filetype is 'tif'
    fileroot = filepath_in.split(os.sep)[-1].replace('.tif', '')
    
    # check to see if an NRRD file for this fileroot exists
    filename = '%s_C0.nrrd' % (fileroot,)
    filepath = os.path.join(dirpath_out, filename)
    if os.path.isfile(filepath):
        print('NRRD file %s already exists' % filepath)
        return fileroot
    
    # load the image (slow)
    im = utils.load(filepath_in)
    
    # crop the ROI
    row, col = roi_top_left
    num_rows, num_cols = roi_shape
    im = im[:, :, row:(row + num_rows), col:(col + num_cols)]
    
    # resample z to obtain isotropic voxels
    z_scale = None
    if z_step_size != pixel_size:
        z_scale = z_step_size / pixel_size
        
    # create an NRRD file for each channel
    for ind in (0, 1):
        filename = '%s_C%d.nrrd' % (fileroot, ind)
        filepath = os.path.join(dirpath_out, filename)
        
        if not write_files:
            continue

        # move the z-dimension from the first to the last dimension
        im_out = np.moveaxis(im[:, ind, :, :], 0, -1)
        im_out = utils.autogain(im_out)
        
        # resample z (note that multichannel=False is required to avoid an error)
        if z_scale:
            print('Resampling z-axis by %s (image shape is %s)' % (z_scale, im_out.shape))
            im_out = skimage.transform.rescale(im_out, (1, 1, z_scale), multichannel=False, preserve_range=True)
            im_out = utils.autogain(im_out)
            
        nrrd.write(filepath, im_out)
        print('Writing %s' % filepath)
        
    return fileroot

In [ ]:
# directory in which to save the NRRD files
nrrd_dir = '../static/demo-data/stacks/'

In [ ]:
# flexo mountpoint
flexo_dir = '/Volumes/MicroscopyData/ML_group/Plate_Microscopy/'

In [ ]:
# write the NRRD files and log the fileroots
all_nrrd_filenames = {}
for target_name, params in ml_fovs_for_demo.items():
    print(target_name)
    
    filepath, top_left_x, top_left_y, z_step_size = params
    
    # x and y are columns and row, respectively
    roi_top_left = (top_left_y, top_left_x)
    
    # full filepath
    filepath = os.path.join(flexo_dir, filepath)
    
    all_nrrd_filenames[target_name] = to_nrrd(filepath, nrrd_dir, roi_top_left, z_step_size, write_files=False)

In [ ]:
with open('../src/demo/data/20190816_nrrd-filepaths.json', 'w') as file:
    json.dump(all_nrrd_filenames, file)

In [ ]:
# debugging: read an NRRD file
data, header = nrrd.read(os.path.join(nrrd_dir, '%s_C0.nrrd' % all_nrrd_filenames['MTOR']))
data.shape, data.dtype

In [ ]:
viz.imshow(data[:, :, 5:65].max(axis=0))

In [ ]:
data, header = nrrd.read(os.path.join(nrrd_dir, '%s_C0.nrrd' % all_nrrd_filenames['ATL2']))
viz.imshow(data[:, :, :].max(axis=0))

### Patch to fix stacks
Crop all stacks so that they have the same number of z-slices. This is a temporary hack to prevent the volume renderer from 'squishing' stacks with more slices than the initial stack. 

In [ ]:
num_slices = 60
def crop_stack(filepath_in, offset=0):
    filepath_out = filepath_in.replace('.nrrd', '_cropped.nrrd')
    data, header = nrrd.read(filepath_in)
    nrrd.write(filepath_out, data[:, :, offset:(offset + num_slices)])

In [ ]:
for target_name, filename in all_nrrd_filenames.items():
    crop_stack(os.path.join(nrrd_dir, '%s_C0.nrrd' % filename))
    crop_stack(os.path.join(nrrd_dir, '%s_C1.nrrd' % filename))

In [ ]:
# custom crops for RAB14, CLTA, LMNA, MTOR
offset = 10
target_name = 'RAB14'
crop_stack(os.path.join(nrrd_dir, '%s_C0.nrrd' % all_nrrd_filenames[target_name]), offset)
crop_stack(os.path.join(nrrd_dir, '%s_C1.nrrd' % all_nrrd_filenames[target_name]), offset)

offset = 15
target_name = 'CLTA'
crop_stack(os.path.join(nrrd_dir, '%s_C0.nrrd' % all_nrrd_filenames[target_name]), offset)
crop_stack(os.path.join(nrrd_dir, '%s_C1.nrrd' % all_nrrd_filenames[target_name]), offset)

offset = 10
target_name = 'LMNA'
crop_stack(os.path.join(nrrd_dir, '%s_C0.nrrd' % all_nrrd_filenames[target_name]), offset)
crop_stack(os.path.join(nrrd_dir, '%s_C1.nrrd' % all_nrrd_filenames[target_name]), offset)

offset = 5
target_name = 'MTOR'
crop_stack(os.path.join(nrrd_dir, '%s_C0.nrrd' % all_nrrd_filenames[target_name]), offset)
crop_stack(os.path.join(nrrd_dir, '%s_C1.nrrd' % all_nrrd_filenames[target_name]), offset)


In [ ]:
filepaths = glob.glob('../static/demo-data/stacks/*.nrrd')
for filepath in filepaths:
    os.rename(filepath, filepath.replace('_cropped.nrrd', '.nrrd'))

### Aside: inspecting a single raw stack

In [ ]:
im = utils.load(filepath)

In [ ]:
# a single z-slice
viz.imshow(im[20, 1, :, :])

In [ ]:
# RGB z-projection (DAPI in gray)
viz.imshow(viz.make_rgb(img=im[:, 1, :, :], im_bg=im[:, 0, :, :]))

In [ ]:
# a single x-z plane
viz.imshow(viz.make_rgb(img=im[:, 1, 0, :], im_bg=im[:, 0, 0, :], gamma=.7))

### Aside: writing the stack as a directory of PNG slices

In [ ]:
fileroot = filepath.split(os.sep)[-1].split('.')[0]
png_dir = os.path.join('%s_PNG' % root, fileroot)
os.makedirs(png_dir, exist_ok=True)
fileroot

In [ ]:
# channels
for channel_ind in range(im.shape[1]):
    im_out = utils.autogain(im[:, channel_ind, :, :])
    
    # z-slices
    for z_ind in range(im.shape[0]):
        s = im_out[z_ind, :, :]
        filename = '%s_C%d_Z%02d.png' % (fileroot, channel_ind, z_ind)
        cv2.imwrite(os.path.join(png_dir, filename), s)